In [ ]:
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
import os
import sys 
sys.path.append('../')
from more_itertools import chunked
import csv

def read_labels(filename):
    files = []
    labels = []
    with open(filename, 'r') as f:
        reader = csv.reader(f)
        next(reader) # skip header
        for row in reader:
            files.append(row[0])
            labels.append(list(chunked(map(int, row[1:]), 2)))
    return files, labels

def imshow(img, size=(5, 5)):
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    plt.rcParams['figure.figsize'] = size
    plt.imshow(img)
    plt.yticks([])
    plt.xticks([])
    plt.show()

files, labels = read_labels("../data/labels/border_labels.csv")
imgs = []
for file in files:
    imgs.append(cv.imread("../" + file))

In [ ]:
warped = []
for img, label in zip(imgs, labels):
    W = min(img.shape[:2])
    imgc = img.copy()

    dest_points = np.array([[0, 0], [0, W], [W, W], [W, 0]], dtype=np.float32)
    M = cv.getPerspectiveTransform(np.array(label, dtype=np.float32), dest_points)
    
    img_warped = cv.warpPerspective(imgc, M, (W, W))
    warped.append(img_warped)

## remove stones and try to inpaint

In [ ]:
imshow(warped[7])

In [ ]:
for warp in warped:
    gray = cv.cvtColor(warp, cv.COLOR_BGR2GRAY)
    filtered = cv.bilateralFilter(gray, 3, 75, 75)
    adaptive = cv.adaptiveThreshold(
        filtered, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY_INV,
        blockSize=15, # to tune
        C=8)          # to tune

    dilated = cv.dilate(adaptive, np.ones((3, 3), np.uint8), iterations=3)
    eroded = cv.erode(dilated, np.ones((3, 3), np.uint8), iterations=3)

    imshow(np.concatenate([gray, eroded], axis=1), size=(20, 20))
